In [2]:
import spectHR as cs
import pyhrv
from ipywidgets import Tab, Output
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('ggplot')
cs.handler.show_logs()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [3]:
DataSet = cs.SpectHRDataset("SUB_018.xdf", use_webdav=True, reset = False)
DataSet = cs.borderData(DataSet)
DataSet = cs.filterECGData(DataSet, {"filterType": "highpass", "cutoff": .50})
if not hasattr(DataSet, 'RTops'):
    DataSet = cs.calcPeaks(DataSet)

In [53]:
%matplotlib widget
GUI = cs.prepPlot(DataSet)

preProcessing =  Output()
with preProcessing:
    display(GUI)

poincarePlot = Output()
psdPlot = Output()
descriptives = Output()
Gantt = Output()

import pandas as pd

App = Tab(children=[preProcessing, poincarePlot, descriptives, psdPlot, Gantt])

App.set_title(0, "PreProcessing")
App.set_title(1, "Poincare")
App.set_title(2, "Descriptives")
App.set_title(3, "PSD")
App.set_title(4, "Epochs")
psd_Values = {}
descriptives_Values = {}

# This function is called everytime a tab is selected in the App. The value of change['new'] holds the index to the selected tab.
def on_tab_change(change):
    if change['name'] == 'selected_index':        
        if change['new'] == 1:  # Poincare selected
            with poincarePlot:
                poincarePlot.clear_output()
                display(cs.poincare(DataSet))
            
        if change['new'] == 2: # selected the descriptives tab
            with descriptives:
                descriptives.clear_output()
                global descriptives_Values
                descriptives_Values = cs.explode(DataSet)\
                        .groupby('epoch')['ibi']\
                        .agg([\
                            ('N', len),\
                            ('mean', 'mean'),\
                            ('std', 'std'),\
                            ('min', 'min'),\
                            ('max', 'max'),\
                            ('rmssd', lambda x: pyhrv.time_domain.rmssd(x)[0]), \
                            ('sdnn', lambda x: pyhrv.time_domain.sdnn(x)[0]),\
                            ('sdsd', cs.Tools.Params.sdsd),\
                            ('sd1', cs.Tools.Params.sd1),\
                            ('sd2', cs.Tools.Params.sd2),\
                            ('sd_ratio', cs.Tools.Params.sd_ratio),\
                            ('ellipse_area', cs.ellipse_area)\
                        ])
                display(descriptives_Values)
                
        if change['new'] == 3:  # psd selected
            with psdPlot:
                psdPlot.clear_output()    
                Data = cs.explode(DataSet)
                global psd_Values
                psd_Values = Data.groupby('epoch')[Data.columns.tolist()].apply(cs.welch_psd)
                display(psd_Parameters)

        if change['new'] == 4:  # gantt selected 
            with Gantt:
                Gantt.clear_output()
                display(cs.gantt(DataSet, labels=False))
                
    DataSet.save()
                
# Attach observer to detect tab switch
App.observe(on_tab_change, names='selected_index')

display(App)

In [55]:
df = pd.DataFrame(list(psd_Parameters.dropna()))
df['epoch'] = psd_Parameters.dropna().index

In [57]:
pd.set_option('display.precision', 12)
df

,VLF Power,LF Power,HF Power,LF/HF Ratio,epoch
0,0.000014780282,0.000251420298,0.000347657417,0.723184046543,bigfive
1,0.000043980340,0.000357479612,0.000362921788,0.985004550247,congruent
2,0.000161963338,0.000609302487,0.000338348922,1.800811078764,incongruent
3,0.000036544825,0.000247432883,0.000411387228,0.601459808145,names
4,0.000060022866,0.000263097865,0.000371830595,0.707574547717,words


In [71]:
pd.set_option('display.precision', 6)
pd.merge(descriptives_Values, df, on = 'epoch', how='outer')

,epoch,N,mean,std,min,max,rmssd,sdnn,sdsd,sd1,sd2,sd_ratio,ellipse_area,VLF Power,LF Power,HF Power,LF/HF Ratio
0,bigfive,247,0.724561,0.028886,0.642334,0.833627,30.724451,28.876328,16.455474,21.733370,34.489616,0.630142,2354.861163,0.000015,0.000251,0.000348,0.723184
1,congruent,554,0.768493,0.033491,0.673113,0.871132,35.091124,33.498857,18.943321,24.807470,40.228637,0.616662,3135.217410,0.000044,0.000357,0.000363,0.985005
2,congruent practice,20,0.773573,0.033193,0.712065,0.829719,32.836116,33.171762,17.797924,23.131878,40.108678,0.576730,2914.735287,NaN,NaN,NaN,NaN
3,incongruent,663,0.753670,0.041515,0.651891,0.913372,33.423267,41.495029,19.396691,23.654395,53.579823,0.441480,3981.649210,0.000162,0.000609,0.000338,1.800811
4,incongruent practice,26,0.751681,0.049282,0.661257,0.848210,40.640374,49.301537,28.637097,28.670625,63.426286,0.452031,5712.896086,NaN,NaN,NaN,NaN
5,names,195,0.748110,0.033933,0.672716,0.837255,35.398614,33.909990,19.051405,25.073863,40.310438,0.622019,3175.328281,0.000037,0.000247,0.000411,0.601460
6,names practice,26,0.760528,0.046556,0.676504,0.870866,35.415251,46.437054,17.984994,25.083559,60.244692,0.416361,4747.421684,NaN,NaN,NaN,NaN
7,words,175,0.741268,0.033224,0.653023,0.824814,30.818620,33.219787,15.543374,21.800172,41.388646,0.526719,2834.594898,0.000060,0.000263,0.000372,0.707575
8,words practice,24,0.733124,0.040147,0.656884,0.812579,35.753382,40.126882,23.227607,25.352478,50.379509,0.503230,4012.584657,NaN,NaN,NaN,NaN
